# Setup

## Install dependencies

In [ ]:
!pip install -qqq langchain-openai langchain-core langchain-community langgraph faiss-cpu

## Create reuqired files

In [9]:
hr_policy_text = """
Leave Policy:
- Annual Leave: 18 days per year. Carry forward up to 6 days.
- Sick Leave: 10 days per year. Medical certificate required beyond 2 days.
- Casual Leave: 6 days per year.

Attendance Policy:
- Working hours: 9:30 AM to 6:30 PM.
- Minimum 8 working hours per day.
- 3 late arrivals per month allowed.

Remote Work Policy:
- Eligible after 6 months tenure.
- Max 2 days per week.
- Not allowed during notice period.

Holiday Policy:
- 10 fixed company holidays per year.

Code of Conduct:
- Professional behavior mandatory.
- No harassment or misuse of company resources.

Exit Policy:
- Notice period: 60 days.
- Buyout subject to HR approval.
"""



In [ ]:
file_path = "/content/hr_policies.txt"

with open(file_path, "w") as f:
    f.write(hr_policy_text)

print("File created at:", file_path)


In [ ]:
# Setup the API Key in to os.environ variables from Colab
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Prepare Data

In [ ]:
# Load the Document
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path)
documents = loader.load()

documents[0].page_content[:300]


In [ ]:
# Split Document into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)

chunks = text_splitter.split_documents(documents)
len(chunks)


In [ ]:
# Create Embeddings & Vector Store (FAISS)
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


# Tools

In [ ]:
# Define HR Tools
from langchain_core.tools import tool
from datetime import datetime

@tool
def classify_hr_policy(query: str) -> str:
    """Classify HR question into policy category."""
    q = query.lower()
    if "leave" in q:
        return "Leave Policy"
    if "remote" in q or "work from home" in q:
        return "Remote Work Policy"
    if "late" in q or "attendance" in q:
        return "Attendance Policy"
    if "notice" in q or "resign" in q:
        return "Exit Policy"
    return "General HR Policy"


@tool
def calculate_days(start_date: str, end_date: str) -> str:
    """Calculate number of days between two dates (YYYY-MM-DD)."""
    s = datetime.strptime(start_date, "%Y-%m-%d")
    e = datetime.strptime(end_date, "%Y-%m-%d")
    return f"{(e - s).days} days"



In [ ]:
def get_hr_context(question: str) -> str:
    docs = retriever.invoke(question)
    return "\n\n".join(d.page_content for d in docs)


In [ ]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

hr_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
You are an HR Policy Assistant.

Strict rules:
- Answer ONLY using HR policy context provided in system messages.
- Do NOT guess or invent policies.
- If information is insufficient, say: "Not defined in policy".
- If the question is ambiguous, ask a clarification question.
- Maintain a professional HR tone.

Always respond using this structure:

Answer:
Source Policy:
Notes:
Confidence Level:
"""),
    MessagesPlaceholder(variable_name="messages")
])


# RAG

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | ChatPromptTemplate.from_messages([
        ("system", "Use the following HR policy context:\n\n{context}"),
        ("human", "{question}")
    ])
    | llm
    | StrOutputParser()
)


In [ ]:
# Create agent

from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[classify_hr_policy, calculate_days],
    prompt=hr_prompt
)


In [ ]:
def ask_hr(question: str):
    context = get_hr_context(question)

    messages = [
        {
            "role": "system",
            "content": f"HR Policy Context:\n{context}"
        },
        {
            "role": "user",
            "content": question
        }
    ]

    return agent.invoke({"messages": messages})


In [ ]:
# Ask a Normal HR Question
result = ask_hr("How many annual leave days do I get?")

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)

In [ ]:
# Clarification Example
query = "Can I apply for remote work?"

result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)


In [ ]:
# Out-of-Scope Question
query = "How is my salary calculated?"

result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)
